In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
# So code is automatically reloaded when saved in different modules

In [2]:
import os
import pickle
import numpy as np
import tensorflow as tf
# from scipy import stats
import tensorflow as tf
# from sklearn.metrics import roc_auc_score
# import sklearn.metrics
from sklearn import metrics
import scipy
import matplotlib.pyplot as plt
import time
import logging
import sys
import time
from datetime import datetime
import csv
from collections import Counter, OrderedDict, defaultdict
import sys
import pprint
sys.path.append('src/taggerSystem/')
from data_util import load_and_preprocess_data, load_embeddings, ModelHelper, lastTrueWordIdxs
logger = logging.getLogger("hw3.q2")
logger.setLevel(logging.DEBUG)
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)

## File Paths


In [3]:
# data_train = "data/icd9NotesDataTable_train.csv"
# data_valid = "data/icd9NotesDataTable_valid.csv"
data_train = "data/smallIcd9NotesDataTable_train.csv"
data_valid = "data/smallIcd9NotesDataTable_valid.csv"
vocab = "src/taggerSystem/data_hw3_delete/vocab.txt"
wordVecs = "src/taggerSystem/data_hw3_delete/wordVectors.txt"
output_path = 'results/model_description'
# log_output = output_path + "log"
# vocab = 'data/icd9Vocab.txt'# vocab for our data
# wordVecs = 'data/newgloveicd9.txt'# len 300 word vectors
output_path = 'results/{}/{:%Y%m%d_%H%M%S}/".format(self.cell, datetime.now())'
log_output = output_path + "log"
START_TOKEN = "<s>"
END_TOKEN = "</s>"
NUM = "NNNUMMM"
UNK = "UUUNKKK"
EMBED_SIZE = 50 # this should not be manually set and should instead come from the word
# vectors. Or maybe it's good that we have to set it so we know for sure the size
# of word vecs we're using
maxAllowedNoteLength = 500
max_grad_norm = 5
codeIdx = 9
textIdx = 6
num_layers = 4

In [4]:
modelRunOutputPath = input(prompt = 'Where should models and performances be saved to?')
if modelRunOutputPath == '':
    modelRunOutputPath = 'results/temp'
if not os.path.exists(modelRunOutputPath):
    os.makedirs(modelRunOutputPath)

Where should models and performances be saved to?


## Extract Data and Word Embeddings
You'll probably only need to work with xDev, yDev and xTrain, yTrain. The X matrices hold all word IDs in the order they appear in the note. yDev is a matrix of indicator vectors for icd9 presence

In [5]:
helper, train, dev, train_raw, dev_raw, xTrain, yTrain, xDev, yDev = load_and_preprocess_data(
    data_train = data_train, data_valid = data_valid, 
    maxAllowedNoteLength = maxAllowedNoteLength, 
    codeIdx = codeIdx, textIdx = textIdx)
embeddings = load_embeddings(vocab, wordVecs, helper, embeddingSize = EMBED_SIZE)
lastTrueWordIdx_train = lastTrueWordIdxs(train)
lastTrueWordIdx_dev = lastTrueWordIdxs(dev)
helper.save(modelRunOutputPath)# token2id and max length saved to output_path
# handler = logging.FileHandler(log_output)
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
# logging.getLogger().addHandler(handler)

In [6]:
np.savetxt(os.path.join(modelRunOutputPath, 'yDev.gz'),yDev)
#np.save(os.path.join(modelRunOutputPath, 'xDev.npy'), xDev)
np.savetxt(os.path.join(modelRunOutputPath, 'devTrueIdxs.gz'), lastTrueWordIdx_dev)
# yDev = np.load(os.path.join(modelRunOutputPath, 'yDev.npy'))
# xDev = np.load(os.path.join(modelRunOutputPath, 'xDev.npy'))
# lastTrueWordIdx_dev = np.load(os.path.join(modelRunOutputPath, 'devTrueIdxs.npy'))

In [7]:
print('Max note length %d'%(helper.max_length))
print('Number of Icd9 codes %d'%(helper.n_labels))
print('icd9 present')
print('xDev shape: nObs = %d, nWords = %d'%(xDev.shape))
print('yDev shape: nObs = %d, nClasses = %d'%(yDev.shape))
print('xTrain shape: nObs = %d, nWords = %d'%(xTrain.shape))
print('yTrain shape: nObs = %d, nClasses = %d'%(yTrain.shape))
print('Embeddings shape: nWords = %d, wordVec len = %d'%(embeddings.shape))
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(helper.icdDict)

Max note length 4
Number of Icd9 codes 6
icd9 present
xDev shape: nObs = 7, nWords = 4
yDev shape: nObs = 7, nClasses = 6
xTrain shape: nObs = 7, nWords = 4
yTrain shape: nObs = 7, nClasses = 6
Embeddings shape: nWords = 13, wordVec len = 50
{'123': 0, '18': 2, '4240': 3, '45': 5, '456': 1, '486': 4}


## Hyperparameters

In [8]:
learning_rate = 0.001
training_epochs = 10
batch_size = 1
total_batches = (xTrain.shape[0]//batch_size)
print('Total number of batches per epoch %d'%(total_batches))
n_input = 1
n_steps = 10
n_hidden = 200
n_classes = helper.n_labels
output_keep_prob = 0.5
input_keep_prob = 1# not sure why there are two but example used 0.5 for output. Check with TA
embeddingSize = embeddings.shape[1]
print('Embedding size is %d'%(embeddingSize))

Total number of batches per epoch 7
Embedding size is 50


In [9]:
# embeddingsSaved = embeddings
# embeddings = embeddings[:, list(range(50))]
# print(embeddingsSaved.shape)
# print(embeddings.shape)
# EMBED_SIZE = 300

In [10]:
# EMBED_SIZE = 300

In [11]:
hyperParamsDict = {'EMBED_SIZE': EMBED_SIZE,
                  'maxNoteLength': maxAllowedNoteLength,
                  'maxGradNorm': max_grad_norm,
                  'outputKeepProb': output_keep_prob,
                  'inputKeepProb': input_keep_prob,
                  'learningRate': learning_rate,
                  'trainingEpochsMax': training_epochs,
                  'batchSize': batch_size,
                  'n_hidden': n_hidden}
with open(os.path.join(modelRunOutputPath, 'hyperParamDict.pickle'), 'wb') as handle:
    pickle.dump(hyperParamsDict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
hyperParamsDict

{'EMBED_SIZE': 50,
 'batchSize': 1,
 'inputKeepProb': 1,
 'learningRate': 0.001,
 'maxGradNorm': 5,
 'maxNoteLength': 500,
 'n_hidden': 200,
 'outputKeepProb': 0.5,
 'trainingEpochsMax': 10}

## Placeholders

In [13]:
tf.reset_default_graph()
x = tf.placeholder(tf.int32, shape= (None, helper.max_length))
yTruth = tf.placeholder(tf.int32, shape = (None, helper.n_labels))
y_steps = tf.placeholder(tf.int32, shape = (None, helper.n_labels))# not sure what this is
trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))# vector which holds true word
outputKeepProb = tf.placeholder(tf.float32, shape=())
inputKeepProb = tf.placeholder(tf.float32, shape=())
# indices of each sentence.

In [14]:
# with tf.Session() as session:
#     nClasses = 2
#     nWords = 2
#     nBatches = 3
#     trueIndices = tf.constant([1,0,1], shape = (nBatches, 1))
#     foo = tf.constant([1,2,3,4,5,6,7,8,9,10,11,12], shape = (nBatches,nWords,nClasses))
#     # foo[:, 1] # [2, 5]
# #     indices = tf.constant([1, 0, 1])
#     print(foo.eval())
    
# #     print('transpose')
# #     foo = tf.transpose(foo,[1,0,2])
# #     print(foo.eval())
    
#     print('flattening')
#     foo = tf.reshape(foo,[-1,nClasses])
#     print(foo.eval())
    
#     print('offset')
#     offset = tf.expand_dims(tf.range(0, nBatches, dtype = tf.int32) * nWords, 1)
#     print(offset.get_shape())
#     print(offset.eval())
#     print('old indices')
#     print(trueIndices.eval())
#     print('new indices')
#     flattened_indices = trueIndices + offset
#     print(flattened_indices.eval())
#     print(flattened_indices.get_shape())
#     print('gathering')
#     print(tf.gather(foo, flattened_indices).eval())
#     print(tf.gather(foo, flattened_indices).get_shape())
#     print('correcting shape')
#     foo = tf.gather(foo, flattened_indices)
#     foo = tf.reshape(foo, [-1, nClasses])
#     print(foo)
#     print(foo.get_shape())
# # foo[:, indexes] # [2, 6]

In [15]:
print(output_keep_prob)
print(input_keep_prob)

0.5
1


## Helper Functions

In [16]:
def LSTM(x, weight, bias, trueWordIdxs, outputKeepProb, inputKeepProb):
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden,state_is_tuple = True)
    cell = tf.contrib.rnn.DropoutWrapper(cell=cell, output_keep_prob = outputKeepProb, 
                                         input_keep_prob = inputKeepProb)
    cell = tf.contrib.rnn.MultiRNNCell(cells=[cell] * num_layers, state_is_tuple=True)
#     If we ever wanna get fancy we can try the above.
    print(type(cell))
    print(cell)
    print(type(x))
    print(x.get_shape())
    print('cell output size')
    print(cell.output_size)
    print('cell state size')
    print(cell.state_size)

    output, state = tf.nn.dynamic_rnn(cell = cell, inputs = x, dtype = tf.float32)
    print('output shape')
    print(output.get_shape())
    
    
    
    
    # new code
    offset = tf.expand_dims(tf.range(0, batch_size, dtype = tf.int32)*helper.max_length, 1)
    offset = tf.expand_dims(tf.range(0, tf.shape(x)[0], dtype = tf.int32)*helper.max_length, 1)
#     offset = tf.expand_dims(tf.range(0, nBatches, dtype = tf.int32) * nWords, 1)
    print('offset shape')
    print(offset.get_shape())
#     1/0
#     foo = tf.reshape(foo,[-1,nClasses])
#     7/0
#     print(tf.shape(output)[2])
#     print(output.get_shape()[2])
    output = tf.reshape(output,[-1, n_hidden]) # collapses the 3d matrix into a 2d
    # matrix where all matrices are stacked on top of eachother
    print('output shape new shape')
    print(output.get_shape())
    flattened_indices = trueWordIdxs + offset
    print('flattened indices shape')
    print(flattened_indices.get_shape())
    output_flattened = tf.gather(output, flattened_indices)
    output_flattened = tf.reshape(output_flattened, [-1, n_hidden])
    print('output flattened shape')
    print(output_flattened.get_shape())
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
    print('output wx + b')
    print(output_logits.get_shape())
    return output_logits




    
    #old code
    output_flattened = tf.gather(tf.transpose(output,[1,0,2]), helper.max_length - 1)
    # should rename to output last really.
    # here is where you're grabbing the last output
    print('output flattened shape')
    print(output_flattened.get_shape())
    output_logits = tf.add(tf.matmul(output_flattened,weight),bias)
# #     flattened_indices = trueIndices + offset
#     print('flattened indices shape')
#     print(flattened_indices.get_shape())
    print('output wx + b')
    print(output_logits.get_shape())
    output_last = output_logits
    print('output last shape')
    print(output_last.get_shape())
    return output_last

In [17]:
with tf.variable_scope('RNN_OutsideCell', reuse = False) as scope:
    U = tf.get_variable(name = 'U', shape = (n_hidden, n_classes), 
                    initializer = tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(name = 'bias', shape = [n_classes], 
                           initializer = tf.constant_initializer(0))
    pretrainedEmbeddings = tf.Variable(embeddings)
    wordEmbeddings = tf.nn.embedding_lookup(params = pretrainedEmbeddings, ids = x)
    print(wordEmbeddings.get_shape())
    print('shape of embeddings')
    print(wordEmbeddings.get_shape())
    print('U shape')
    print(U.get_shape())
    print('bias shape')
    print(bias.get_shape())
    y_last = LSTM(wordEmbeddings,U,bias, trueWordIdxs, outputKeepProb, inputKeepProb)# TODO is y_last the correct thing to return?
    print(y_last.get_shape())


(?, 4, 50)
shape of embeddings
(?, 4, 50)
U shape
(200, 6)
bias shape
(6,)
<class 'tensorflow.contrib.rnn.python.ops.core_rnn_cell_impl.MultiRNNCell'>
<class 'tensorflow.python.framework.ops.Tensor'>
(?, 4, 50)
cell output size
200
cell state size
(LSTMStateTuple(c=200, h=200), LSTMStateTuple(c=200, h=200), LSTMStateTuple(c=200, h=200), LSTMStateTuple(c=200, h=200))
output shape
(?, 4, 200)
offset shape
(?, 1)
output shape new shape
(?, 200)
flattened indices shape
(?, 1)
output flattened shape
(?, 200)
output wx + b
(?, 6)
(?, 6)


## Loss Function

In [18]:
batchError = tf.nn.sigmoid_cross_entropy_with_logits(logits = y_last, 
                                                 labels = tf.cast(yTruth, tf.float32))
loss_function = tf.reduce_mean(batchError)



# hw3 way
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate) dis from hw3
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss_function, tvars), max_grad_norm)
gradientVars = zip(grads, tvars)
train_op = optimizer.apply_gradients(gradientVars)

# rando way
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# gvs = optimizer.compute_gradients(cost)
# capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
# train_op = optimizer.apply_gradients(capped_gvs)


# train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_function)

/home/oliver/anaconda3/envs/clinicalNoteTagger/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## Training and Testing

In [19]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
# hacky work around for issue where -1 is padding but now maps to UUNNNKKK vector.

In [20]:
epochAvgLoss = np.zeros(training_epochs)
epochAvgLossValid = np.zeros(training_epochs)
validLossIncreasingCount = 0
maxIncreasingLossCount = 3
prevValidLoss = np.inf
minValidLoss = np.inf

In [21]:
epochPredictions = np.zeros(shape = [training_epochs, yDev.shape[0], yDev.shape[1]])

In [22]:
minValidLoss = np.inf

In [23]:
# tf.reset_default_graph()
# x = tf.placeholder(tf.int32, shape= (None, helper.max_length))
# yTruth = tf.placeholder(tf.int32, shape = (None, helper.n_labels))
# y_steps = tf.placeholder(tf.int32, shape = (None, helper.n_labels))# not sure what this is
# trueWordIdxs = tf.placeholder(tf.int32, shape = (None,1))# vector which holds true word
# outputKeepProb = tf.placeholder(tf.float32, shape=())
# inputKeepProb = tf.placeholder(tf.float32, shape=())

In [25]:
all_saver = tf.train.Saver()
tf.add_to_collection('y_last', y_last)
tf.add_to_collection('x', x)
tf.add_to_collection('trueWordIdxs', trueWordIdxs)
tf.add_to_collection('outputKeepProb', outputKeepProb)
tf.add_to_collection('inputKeepProb', inputKeepProb)
tf.add_to_collection('pretrainedEmbeddings', pretrainedEmbeddings)
with open(os.path.join(modelRunOutputPath, 'runOutput.txt'), 'w') as f:
    with tf.Session() as session:
#         all_saver.restore(session, 'tempDelete/bestModel.data-00000-of-00001')
        tf.global_variables_initializer().run()
        for epoch in range(training_epochs):
            totalError = 0.0
            f.write("""***************************
***************************
Running on epoch %d
***************************
***************************
***************************\n""" %(epoch))
            print('***************************')
            print('***************************')
            print('Running on epoch %d'% (epoch))
            print('***************************')
            print('***************************')
            start = time.time()
            for b in range(total_batches):
#             for b in range(1):
#             for b in range(total_batches + 1):
#                 offset = (b * batch_size) % (yTrain.shape[0] - batch_size)
                offset = min((b * batch_size), yTrain.shape[0])
                batch_x = xTrain[offset:(offset + batch_size), :]
                batch_y = yTrain[offset:(offset + batch_size), :]
                batchTrueWordIdxs = lastTrueWordIdx_train[offset:(offset + batch_size)]
#                 print(batch_x)
#                 print(batchTrueWordIdxs)
#                 print(batchTrueWordIdxs.shape)
#                 print(batch_y)
#                 print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
                _, c = session.run([train_op, loss_function],feed_dict={x: batch_x, 
                                                                        yTruth: batch_y, 
                                                                        trueWordIdxs: batchTrueWordIdxs,
                                                                       outputKeepProb: output_keep_prob, 
                                                                        inputKeepProb: input_keep_prob})
                if(b%25 == 0):
                    f.write('running iteration %d with loss %3f \n'% (b, c))
                    print('running iteration %d with loss %3f'% (b, c))
                totalError = totalError + c
#             print('here we go')
#             smallerSizeForComp = 13181
#             smallerSizeForComp = 6590
#             smallerSizeForComp = 3295
            batchSizeDev = 3295
            totalBatchesDev = (xDev.shape[0]//batchSizeDev)
            pred_y = np.full(shape = yDev.shape, fill_value = -1.0, dtype = np.float32)
            for b in range(totalBatchesDev + 1):
#                 print(b)
                offset = min((b * batchSizeDev), yDev.shape[0])
                devBatch_x = xDev[offset:(offset + batchSizeDev), :]
                devBatchTrueWordIdxs = lastTrueWordIdx_dev[offset:(offset + batchSizeDev)]
                pred_yBatch = session.run(y_last,feed_dict={x: devBatch_x,
                                                      trueWordIdxs:devBatchTrueWordIdxs,
                                                      outputKeepProb: 1,
                                                        inputKeepProb: 1}, ) # must be set to one for predictions.
                pred_y[offset:(offset + batchSizeDev), :] = pred_yBatch
#             print(pred_y.shape)
            if (pred_y == -1).all():
                print('negative values exist. This means indexing is off in pred_yBatch')
                1/0
#             print('got preds')
            validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                                 labels = tf.cast(yDev, tf.float32))
#             print('validLoss')
            validLoss = tf.reduce_mean(validLoss)
            validLoss = validLoss.eval()
            epochPredictions[epoch,:,:] = pred_y
            epochAvgLossValid[epoch] = validLoss
            epochAvgLoss[epoch] = totalError/total_batches
            print('average loss %f'% (totalError/total_batches))
            print('test loss %f'%(validLoss))
#             print('previous valid loss %f'%(prevValidLoss))
            print('Total run time was %3f'% (time.time() - start))
            f.write('average loss %f \n'%(totalError/total_batches)) 
            f.write('test loss %f \n'%(validLoss))
            f.write('Total run time was %3f \n'% (time.time() - start))
            if validLoss <= minValidLoss:
                validLossIncreasingCount = 0
                minValidLoss = validLoss
                print('New best model found. Saving')
                all_saver.save(session, os.path.join(modelRunOutputPath, 'bestModel'))
                # save model
            if validLoss > prevValidLoss:
                print('validation Loss Increase')
                validLossIncreasingCount += 1
#             print('increasing loss count %d'%(validLossIncreasingCount))
            prevValidLoss = validLoss
            if validLossIncreasingCount == maxIncreasingLossCount:
                print('Stopping early because of increasing validation loss')
                f.write('Stopping early because of increasing validation loss')
                break
#             if(epoch%10 == 0):
#                 all_saver.save(session, os.path.join(modelRunOutputPath, 'checkpointModel'), 
#                                global_step = epoch)
#             print(epochAvgLossValid[0:(epoch+1)])
#             print(epochAvgLoss[0:(epoch+1)])
lastEpoch = epoch

***************************
***************************
Running on epoch 0
***************************
***************************
running iteration 0 with loss 0.693352
average loss 0.687686
test loss 0.676242
Total run time was 0.517650
New best model found. Saving
***************************
***************************
Running on epoch 1
***************************
***************************
running iteration 0 with loss 0.666012
average loss 0.639375
test loss 0.600651
Total run time was 0.465339
New best model found. Saving
***************************
***************************
Running on epoch 2
***************************
***************************
running iteration 0 with loss 0.560708
average loss 0.530206
test loss 0.445092
Total run time was 0.365347
New best model found. Saving
***************************
***************************
Running on epoch 3
***************************
***************************
running iteration 0 with loss 0.242976
average loss 0.508250
test

In [24]:
xDev[xDev == -1] = 0
xTrain[xTrain == -1] = 0
with tf.Session() as session:
#         all_saver.restore(session, 'tempDelete/bestModel.data-00000-of-00001')
    tf.global_variables_initializer().run()
    for _ in range(training_epochs):
        _, trainingError = session.run([train_op, loss_function],
                           feed_dict={x: xTrain, 
                                        yTruth: yTrain, 
                                        trueWordIdxs: lastTrueWordIdx_train,
                                        outputKeepProb: output_keep_prob, 
                                        inputKeepProb: input_keep_prob})
        pred_y = session.run(y_last,feed_dict={x: xDev,
                                      trueWordIdxs:lastTrueWordIdx_dev,
                                      outputKeepProb: 1,
                                      inputKeepProb: 1}, ) # must be set to one for predictions.
        validLoss = tf.nn.sigmoid_cross_entropy_with_logits(logits = pred_y, 
                                             labels = tf.cast(yDev, tf.float32))
        validLoss = tf.reduce_mean(validLoss)
        validLoss = validLoss.eval()
        print('train loss %f'% (trainingError))
        print('test loss %f'%(validLoss))
        print('***********************************************')

train loss 0.692554
test loss 0.688521
***********************************************
train loss 0.688572
test loss 0.683234
***********************************************
train loss 0.684168
test loss 0.675837
***********************************************
train loss 0.675975
test loss 0.664854
***********************************************
train loss 0.662930
test loss 0.648139
***********************************************
train loss 0.654467
test loss 0.623220
***********************************************
train loss 0.634896
test loss 0.587198
***********************************************
train loss 0.600656
test loss 0.540134
***********************************************
train loss 0.573571
test loss 0.490091
***********************************************
train loss 0.524332
test loss 0.454270
***********************************************
